# BP

In this notebook, we take a deep dive into the details of backward propagation (more commonly called backpropagation). To convey some insight for both the techniques and their implementations, we rely on some basic mathematics and computational graphs. To start, we focus our exposition on a one-hidden-layer MLP with weight decay (L2 regularization).


### Forward propagation
Forward propagation (or forward pass) refers to the calculation and storage of intermediate variables (including outputs) for a neural network in order from the input layer to the output layer. We now work step-by-step through the mechanics of a neural network with one hidden layer. 

For the sake of simplicity, let us assume that the input example is $\mathbf x ∈ \mathbb R^d$ and that our hidden layer does not include a bias term. Here the intermediate variable is:

$$ \mathbf z = \mathbf W^{(1)} \mathbf x,$$

where $\mathbf W^{(1)}\in \mathbb R^{h\times d}$ is the weight parameter of the hidden layer. After running the intermediate variable $ \mathbf z ∈ \mathbb R^h$ through the activation function $\phi$ we obtain our hidden activation vector of length $h$,

$$ \mathbf h = \phi(\mathbf z).$$

The hidden variable $h$ is also an intermediate variable. Assuming that the parameters of the output layer only possess a weight of $\mathbf W^{(2)} \in \mathbb R^{q\times h}, we can obtain an output layer variable with a vector of length $q$:

$$ \mathbf o = \mathbf W^{(2)} \mathbf h.$$

Assuming that the loss function is $l$ and the example label is $y$, we can then calculate the loss term
for a single data example,

$$L = l(\mathbf o, y).$$

According to the definition of $L_2$ regularization, given the hyperparameter $\lambda$, the regularization
term is

$$ s = \frac{\lambda}{2}\left(|| \mathbf W^{(1)}||^2_F + \mathbf W^{(2)}||^2_F  \right), $$

where the Frobenius norm of the matrix is simply the $L_2$ norm applied after flattening the matrix into a vector. Finally, the modelʼs regularized loss on a given data example is:

$$J = L + s. $$

We refer to $J$ as the *objective function* in the following discussion.

### Computational graphof forward propagation
We use computational graphs below to visualize the dependencies of operators and variables within the calculation. The lower-left corner signifies the input and the upper-right corner is the output. Notice that the directions of the arrows (which illustrate data flow) are primarily rightward and upward.
<div>
<img src="img\Computation_graph_forward_propagation.png" width="450"/>
</div>

### Backpropagation
Backpropagation refers to the method of calculating the gradient of neural network parameters. In short, the method traverses the network in reverse order, from the output to the input layer, according to the **chain rule** from calculus. The algorithm stores any intermediate variables (partial derivatives) required while calculating the gradient with respect to some parameters.

Continuing the example above, the parameters of the simple network with one hidden layer, whose computational graph is above, are $\mathbf W^{(1)}$ and $\mathbf W^{(2)}$. The objective of backpropagation is to calculate the gradients 

$$\frac{\partial J}{\partial \mathbf W^{(1)}} \text{  and  } \frac{\partial J}{\partial \mathbf W^{(2)}}$$

To accomplish this, we apply the chain rule and calculate, in turn, the gradient of each intermediate variable and parameter. The order of calculations are reversed relative to those performed in forward propagation, since we need to start with the outcome of the computational graph and work our way towards the parameters. The first step is to calculate the gradients of the objective function $J = L + s$ with respect to the loss term $L$ and the regularization
term $s$.

$$\frac{\partial J}{\partial L} =1 \text{  and  }  \frac{\partial J}{\partial s} = 1$$

Next, we compute the gradient of the objective function with respect to variable of the output layer $\mathbf o$ according to the chain rule:

$$\frac{\partial J}{\partial \mathbf o} = \text{prod}\left( \frac{\partial J}{\partial L}, \frac{\partial L}{\partial \mathbf o}\right)=\frac{\partial L}{\partial \mathbf o} \in \mathbb R^q.$$

Next, we calculate the gradients of the regularization term with respect to both parameters:

$$ \frac{\partial s}{\partial \mathbf W^{(1)}}=\lambda \mathbf W^{(1)} \text{  and  }
   \frac{\partial s}{\partial \mathbf W^{(2)}}=\lambda \mathbf W^{(2)}
$$

### 1. $\frac{\partial J}{\partial \mathbf W^{(2)}}$

Now we are able to calculate the gradient ${\partial J}/{\partial \mathbf W^{(2)}} \in \mathbb R^{q\times h}$ of the model parameters closest to the output layer. Using the chain rule yields:

$$ \frac{\partial J}{\partial \mathbf W^{(2)}} = 
   \text{prod}\left( \frac{\partial J}{\partial \mathbf o}, \frac{\partial \mathbf o}{\partial \mathbf W^{(2)}}\right) + 
   \text{prod}\left( \frac{\partial J}{\partial s}, \frac{\partial s}{\partial \mathbf W^{(2)}}\right) =
   \frac{\partial J}{\partial \mathbf o}\mathbf h^T + \lambda\mathbf W^{(2)}.
$$

### 2. $\frac{\partial J}{\partial \mathbf W^{(1)}}$

Go back further, we should calculate the gradient of $J$ with respect to  $\mathbf W^{(1)}$. To obtaine it we need to continue backpropagation along the output layer to the hidden layer. The gradient with respect to the hidden layerʼs outputs ${\partial J}/{\partial \mathbf h} \in \mathbb R^{h}$ is given by

$$ \frac{\partial J}{\partial \mathbf h}=
\text{prod}\left( \frac{\partial J}{\partial \mathbf o}, \frac{\partial \mathbf o}{\partial \mathbf h} \right) =
{\mathbf W^{(2)}}^T \frac{\partial J}{\partial \mathbf o}.
$$

Since the activation function $\phi$ applies elementwise, calculating the gradient ${\partial J}/{\partial \mathbf z} \in \mathbb R^{h}$ of the intermediate variable $\mathbf z$ requires that we use the elementwise multiplication operator, which we denote by $\odot$:

$$ \frac{\partial J}{\partial \mathbf z}=
\text{prod}\left( \frac{\partial J}{\partial \mathbf h}, \frac{\partial \mathbf h}{\partial \mathbf z} \right) =
\frac{\partial J}{\partial \mathbf h}\odot\phi'(\mathbf z).
$$

Finally, we can obtain the gradient ${\partial J}/{\partial \mathbf W^{(1)}} \in \mathbb R^{h\times d}$ of the model parameters closest to the input layer. According to the chain rule, we get

$$ \frac{\partial J}{\partial \mathbf W^{(1)}} = 
   \text{prod}\left( \frac{\partial J}{\partial \mathbf z}, \frac{\partial \mathbf z}{\partial \mathbf W^{(1)}}\right) + 
   \text{prod}\left( \frac{\partial J}{\partial s}, \frac{\partial s}{\partial \mathbf W^{(1)}}\right) =
   \frac{\partial J}{\partial \mathbf z}\mathbf x^T + \lambda\mathbf W^{(1)}.
$$

### Training neural networks
When training neural networks, forward and backward propagation depend on each other. In particular, for forward propagation, we traverse the computational graph in the direction of de- pendencies and compute all the variables on its path. These are then used for backpropagation where the compute order on the graph is reversed.

Note that backpropagation reuses the stored intermediate values from forward propagation to avoid duplicate calculations. One of the consequences is that we need to retain the intermediate values until backpropagation is complete. This is also one of the reasons why training requires significantly more memory than plain prediction. Besides, the size of such intermediate values is roughly proportional to the number of network layers and the batch size. Thus, training deeper networks using larger batch sizes more easily leads to out of memory errors.